In [1]:
from ete3 import Tree
from ete3 import TreeStyle
import os
import matplotlib as plt
from Bio import Phylo

In [ ]:
TREEDIR="/data/zhaolian/LineageTracing/DSS/PacBio/CCS5passes/8.tree/"
sample="5N"
NW=TREEDIR+sample+".phy.treefile"
PHY = TREEDIR+sample+".phy"
tree = Tree(NW)
print(tree)

In [13]:
t1 = Phylo.read(NW, 'newick')
terminal_names = [i.name for i in t1.get_terminals()]
# depth = [t1.depths()[t1.find_any(name=i)] for i in terminal_names]

In [14]:
terminal_names[:10]

['ref',
 'C2',
 'C37824',
 'C40054',
 'C5302',
 'C25345',
 'C3749',
 'C18121',
 'C38142',
 'C29693']

In [21]:
import numpy as np
import pandas as pd
from Bio import Phylo
# import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression


In [61]:
samples= ["5N","5T","16N","16T"]
INDIR="/data/zhaolian/LineageTracing/DSS/PacBio/CCS5passes/8.tree/"
OUTDIR="/data/zhaolian/LineageTracing/DSS/PacBio/CCS5passes/9.tree_manipulation/"
if not os.path.exists(OUTDIR):
    os.mkdir(OUTDIR)
for mySample in samples:
    print(mySample)
    mut_nums = pd.read_csv(INDIR + mySample+".phy", delimiter = " ", skiprows=1,header = None, names = ["name","bi"], index_col=0)
    mut_nums["numMuts"] = [mut_nums["bi"][i].count("1") for i in range(mut_nums.shape[0])]
    tree = Phylo.read(INDIR+mySample+'.phy.treefile', "newick")
    tree_depths = tree.depths()
    depths = []
    mutations = []
    for i in tree_depths:
        if i.name in mut_nums.index:
            depths.append(tree_depths[i])
            mutations.append(mut_nums.loc[i.name].numMuts)
    depths = np.array(depths)
    mutations = np.array(mutations)
    mutations

    reg = LinearRegression(fit_intercept=False).fit(depths.reshape(-1,1), mutations)
    scale = reg.coef_[0]
    print(scale)

    for clade in tree.get_terminals():
        clade.branch_length = float(scale * clade.branch_length)
    for clade in tree.get_nonterminals()[1:]:
        clade.branch_length = float(scale * clade.branch_length)

    Phylo.write(tree, OUTDIR+mySample+'_rescale.nwk','newick')

5N
1493.3460981625215
5T
783.2961791765156
16N
1352.2123401041758
16T
869.6343330261255


In [55]:
os.chdir(OUTDIR)

In [58]:
import numpy as np
import pandas as pd
from Bio import Phylo
# import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

mut_nums = pd.read_csv('5N_phy_numMuts.txt', delimiter='\t', index_col=1)
tree = Phylo.read('5N.phy.treefile', "newick")
tree_depths = tree.depths()
depths = []
mutations = []
for i in tree_depths:
    if i.name in mut_nums.index:
        depths.append(tree_depths[i])
        mutations.append(mut_nums.loc[i.name].numMuts)
depths = np.array(depths)
mutations = np.array(mutations)
reg = LinearRegression(fit_intercept=False).fit(depths.reshape(-1,1), mutations)
scale = reg.coef_[0]
print(scale)
for clade in tree.get_terminals():
    clade.branch_length = float(scale * clade.branch_length)
for clade in tree.get_nonterminals()[1:]:
    clade.branch_length = float(scale * clade.branch_length)
    
Phylo.write(tree, 'tree.nwk','newick')

1493.3460981625215


1

In [20]:
depth

2838

In [80]:
tree.prune(terminal_names)

In [81]:
tree.write(format=1, outfile=sample+"_pruned"+".nw")

In [63]:
"C77515" in terminal_names

False

In [65]:
len(depth)

2474

In [31]:
dir(tree)
tree.children

[Tree node 'C77515' (0x7f5683e9131)]